In [1]:
from PyLQR.sim import KDLRobot
from PyLQR.system import PosOrnPlannerSys, PosOrnKeypoint
from PyLQR.solver import BatchILQRCP, BatchILQR, ILQRRecursive
from PyLQR.utils import primitives, PythonCallbackMessage

import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm

import logging
import sys

from scipy.spatial.transform import Rotation 

from cv_bridge import CvBridge, CvBridgeError

from utils.camera_utils import RealCamera, RealCameraROS
from utils.transform_utils import *
from utils.iLQR_wrapper import iLQR
from utils.visualisation_utils import draw_triangle, draw_box
from utils.views_utils import *
from utils.ROS_utils import generate_grasps_client, format_pointcloud_msg, run_action, get_camera_pose, gridRegistrator

import argparse
from scipy.spatial.transform import Rotation

import time

from contact_grasp.srv import contactGraspnetPointcloud2, contactGraspnetPointcloud2Response

import json
import rospy
from cv_bridge import CvBridge, CvBridgeError
from sensor_msgs import point_cloud2
from sensor_msgs.msg import PointField, CameraInfo, Image, PointCloud2
from std_msgs.msg import Header
from geometry_msgs.msg import PoseArray, Pose
import message_filters

import open3d as o3d

import rospy
from cv_bridge import CvBridge, CvBridgeError
from sensor_msgs.msg import Image
from contact_grasp.srv import segmentationSrv, segmentationSrvResponse
from utils.camera_utils import RealCamera, RealCameraROS

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2023-07-03 15:41:09,907 - utils - NumExpr defaulting to 8 threads.
INFO - 2023-07-03 15:41:11,507 - base_toolkit - Could not load plugin 'qt' from 'pyface.ui.qt.init'
INFO - 2023-07-03 15:41:11,508 - base_toolkit - No pyface.toolkits plugin could be loaded for qt
INFO - 2023-07-03 15:41:11,509 - base_toolkit - Could not load traitsui.toolkits plugin 'qt' from 'traitsui.qt4'
INFO - 2023-07-03 15:41:11,512 - base_toolkit - Could not load traitsui.toolkits plugin 'wx' from 'traitsui.wx'
INFO - 2023-07-03 15:41:11,672 - base_toolkit - Could not load plugin 'qt4' from 'pyface.ui.qt.init'
INFO - 2023-07-03 15:41:11,691 - base_toolkit - No pyface.toolkits plugin could be loaded for qt4
INFO - 2023-07-03 15:41:11,698 - base_toolkit - Could not load traitsui.toolkits plugin 'qt4' from 'traitsui.qt4'


# Connect to camera

In [26]:
bridge = CvBridge()
try:
    rospy.init_node("python_node",anonymous=True)
except:
    print("rospy already initialized")

# dispose_pos = np.array([dim, 0.66, 0.1])
# dispose_orn_wxyz = np.array([0, 1, 0.35, 0])

camera_connexion = "ROS"
if camera_connexion == "ROS":
    camera = RealCameraROS()
    intrinsic, distortion = camera.getIntrinsic()
elif camera_connexion == "pyWrapper":
    camera = RealCamera()
    camera.start()
    intrinsic, distortion = camera.getIntrinsic()
    #retrieve image and depth to initialise camera, otherwise image is very dark
    for i in range(15):
        rgb, depth_image, depth_scale = camera.get_rgb_depth()
else:
    raise Exception("Please choose a valid camera connexion method: ROS or pyWrapper")

# sys.path.append("/home/vdrame/catkin_ws/src/py_panda/PyPanda")
from PyPanda import Robot
from PyPanda import Utils
rbt = Robot("panda", use_gripper=True)
current_pose = get_camera_pose(rbt, ee_depth=-0.10340-0.005)


Camera topic found


# Get image, segmap and viewpoint

In [27]:
#load pos and orn from json file
with open('config/views_pos.json') as json_file:
    views = json.load(json_file)

pos_dif = 1000
keys = views.keys()
angle_range = [-140, 40]
pc_fused = []
pc_colors_fused = []
reference_pose = np.eye(4)
plot_coordinate = []
# while (not detected or not detected_with_collision):
horizon = 30

# get pc
bgr_cv, depth_cv, depth_scale = camera.get_rgb_depth()
depth_cv = depth_cv * depth_scale
rgb_cv = cv2.cvtColor(bgr_cv, cv2.COLOR_BGR2RGB)
pc_fused, pc_colors_fused = depth2pc(depth_cv, intrinsic, rgb_cv)
pc_fused, pc_colors_fused = regularize_pc(pc_fused, pc_colors_fused, 
                                            downsampling_method="voxel", voxel_size=0.005,
                                            outlier_filtering_method="radius", radius_param_arg=[25, 0.015])


pc_fused = current_pose @ np.concatenate((pc_fused, np.ones((pc_fused.shape[0], 1))), axis=1).T
pc_fused = pc_fused.T[:, :3]

#origin
origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.2)

#draw camera coordinate
coordinate = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1).transform(current_pose)
coordinate_line = draw_triangle(color=[1,0,0]).transform(current_pose)
plot_coordinate.append(coordinate)
plot_coordinate.append(coordinate_line)

#draw pc
pc_o3d = o3d.geometry.PointCloud()
pc_o3d.points = o3d.utility.Vector3dVector(pc_fused)
pc_o3d.colors = o3d.utility.Vector3dVector(pc_colors_fused[:, [2,1,0]]/255)
coordinate = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)
pc_o3d.transform(reference_pose)
to_draw = plot_coordinate.copy()
to_draw.append(pc_o3d)
to_draw.append(origin)

o3d.visualization.draw_geometries(to_draw)

Downsampling time: 1.9128029346466064s, Filtering time: 0.19800305366516113s


In [28]:
bridge = CvBridge()
try:
    rospy.init_node("python_node",anonymous=True)
except:
    print("rospy already initialized")

bgr = cv2.cvtColor(rgb_cv, cv2.COLOR_RGB2BGR)
#load cv image
# img = cv2.imread("/home/vdrame/bgr.png")

#convert to ros image
ros_img = bridge.cv2_to_imgmsg(bgr, encoding="bgr8")

#call service
# rospy.wait_for_service("segmentation")

try:
    segmentation = rospy.ServiceProxy("segmentation", segmentationSrv)
    resp = segmentation(ros_img)
except rospy.ServiceException as e:
    print(e)

#convert to cv image
segmap = bridge.imgmsg_to_cv2(resp.image, desired_encoding="passthrough")
print(np.unique(segmap))
#show image
cv2.imshow("segmented image", segmap)
cv2.imshow("rgb", rgb_cv)
key=cv2.waitKey(0)

cv2.destroyAllWindows()

[0. 1. 2. 3.]


In [29]:
# Compute ROI
pc_ROI, pc_colors_ROI, bounding_point = get_ROI_box(pc_fused, pc_colors_fused, depth_cv, rgb_cv, segmap, intrinsic, current_pose, border_size = 0.0)


In [30]:

compute_views_scores(views, pc_fused, pc_colors_fused, intrinsic, bounding_point, image_size=(720, 1280), show=True)


('view1', 0.1621701388888889)

In [31]:
# PC and bounding box for visualization
pc_o3d = o3d.geometry.PointCloud()
pc_o3d.points = o3d.utility.Vector3dVector(pc_fused)
pc_o3d.colors = o3d.utility.Vector3dVector(pc_colors_fused[:, [2,1,0]]/255)

bounding_point_o3d = draw_box(bounding_point[:, :3]) # o3d.utility.Vector3dVector(bounding_point[:, :3])
bounding_point_o3d.paint_uniform_color([1,0,0])

to_draw = plot_coordinate.copy()
to_draw.append(pc_o3d)
to_draw.append(bounding_point_o3d)

# Draw camera coordinate
for view_key in views:
    viewTransform = np.eye(4)
    viewTransform[0:3, 3] = views[view_key]["pos"]
    viewTransform[0:3, 0:3] = Rotation.from_quat(np.array(views[view_key]["orn_wxyz"])[[1, 2, 3, 0]]).as_matrix()

    # coordinate = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1).transform(current_pose)
    view_coordinate = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)
    view_coordinate.transform(viewTransform)

    view_coordinate_line = draw_triangle(color=[0, 0, 1]).transform(viewTransform)

    to_draw.append(view_coordinate)
    to_draw.append(view_coordinate_line)

o3d.visualization.draw_geometries(to_draw)

pc_o3d = o3d.geometry.PointCloud()
pc_o3d.points = o3d.utility.Vector3dVector(pc_ROI)
pc_o3d.colors = o3d.utility.Vector3dVector(pc_colors_ROI[:, [2,1,0]]/255)

bounding_point_o3d = draw_box(bounding_point[:, :3]) # o3d.utility.Vector3dVector(bounding_point[:, :3])
bounding_point_o3d.paint_uniform_color([1,0,0])

# coordinate.translate([0,0 , 0.4])
# pc_o3d.transform(current_pose)
to_draw = plot_coordinate.copy()
to_draw.append(pc_o3d)
to_draw.append(bounding_point_o3d)

viewTransform = np.eye(4)
viewTransform[0:3, 3] = views[next_view]["pos"]
viewTransform[0:3, 0:3] = Rotation.from_quat(np.array(views[next_view]["orn_wxyz"])[[1, 2, 3, 0]]).as_matrix()

# coordinate = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1).transform(current_pose)
view_coordinate = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)
view_coordinate.transform(viewTransform)

view_coordinate_line = draw_triangle(color=[0, 0, 1]).transform(viewTransform)

to_draw.append(view_coordinate)
to_draw.append(view_coordinate_line)

o3d.visualization.draw_geometries(to_draw)

NameError: name 'next_view' is not defined